In [1]:
import graph_characteristics_lib as gc

In [3]:
def bernoulli(p):
    x = randint(0,10^6)
    if x < p*10^6:
        return 1
    else:
        return 0
#change the set V for large graphs
def erdos_renyi_temp(n,tm,p, dire = True):
    g = set()
    V = list( [str(i) for i in range(n)] )
    for t in range(1,tm+1):
        for (a,b) in Combinations(V,2):
            if bernoulli(p):
                if bernoulli(0.5):
                    g.add((a,b,t))
                    if not dire:
                        g.add((b,a,t))
                        
                else:
                    g.add((b,a,t))
                    if not dire:
                        g.add((a,b,t))
                        
                    
    return g

In [16]:
import numpy as np
def temp_to_matrices(g):
    no = gc.nodes(g)
    l_nodes = list(no)
    l_nodes.sort()
    t_max = max(gc.events(g))
    res = [  np.matrix([ [0 for i in range(len(no))] for j in range(len(no))  ]) for ii in range(t_max)   ]
    for (a,b,t) in g:
        res[t-1][l_nodes.index(a),l_nodes.index(b)] = 1
        res[t-1][l_nodes.index(b),l_nodes.index(a)] = 1
    return res
from numpy.linalg import inv,eig
import functools
def mats_to_q(mats,alpha=-1):
    rho = max([ max(list(eig(e)[0]))  for e in mats ])
    if alpha == -1:
        alpha = 1/(2*rho)
    id_mat = np.matrix( [   [0 if i!=j else 1 for j in range(len(mats[0])) ] for i in range(len(mats[0]))  ])
    return functools.reduce(np.matmul,list(map( lambda x : inv(id_mat-alpha*x), mats )))

def mats_to_q2(mats,alpha=-1):
    rho = max([ max(list(eig(e)[0]))  for e in mats ])
    if alpha == -1:
        alpha = 1/(2*rho)
    id_mat = np.matrix( [   [0 if i!=j else 1 for j in range(len(mats[0])) ] for i in range(len(mats[0]))  ])
    res = id_mat
    for i in range(len(mats)):
        res = np.matmul(res, mats[i])        
    return res

def mats_eigen(mats):
    return functools.reduce(np.matmul,list(map( lambda x : x.transpose(), mats )))

def mats_auth(mats):
    return functools.reduce(np.matmul,list(map( lambda x : x.transpose()*x, mats )))

def mats_hubs(mats):
    return functools.reduce(np.matmul,list(map( lambda x : x*x.transpose(), mats )))

In [17]:
n = 10
T = 7
p = 5/(n*T)

In [19]:
g = erdos_renyi_temp(n,T,p,dire = False)
mats = temp_to_matrices(g)
q = mats_to_q(mats)
_, col, _, _ = gc.weisfeiler_lehman_graph_hash(g)

begin all neigbour
end all neigbour
iteration 2
iteration 3
iteration 4
